# Global Model Testing (v3.1)

Casey A Graff

November 30th, 2017

Now using updated model evaluation pipeline.

In [ ]:
# === IMPORTS ===

REP_DIR = "/home/cagraff/Documents/dev/fire_prediction/"
SRC_DIR = REP_DIR + 'src/'
DATA_DIR = REP_DIR + 'data/'

# --- System imports
import os
import numpy as np
from matplotlib import pyplot as plt
from collections import defaultdict

# --- Package imports
os.chdir(SRC_DIR)
import evaluation.metrics as metrics
import evaluation.evaluate_model as evm
import visualization.mapping as vmap
import visualization.stats as vstat

import helper.geometry as geo

from models import linear_regression as lr
from models import poisson_regression as pr
from models import grid_predictor as gp
from models import bias_grid as bg
from models import active_ignition_grid as aig


# Setup mpl
plt.rcParams['figure.figsize'] = [7,5]
%matplotlib inline

In [ ]:
# === DATA LOADING ===

T_K_MAX = 5
T_K_ARR = np.arange(1,T_K_MAX+1)

# --- Data files
ignition_cube_src = os.path.join(DATA_DIR, 'interim/modis/fire_cube/fire_ignition_cube_modis_alaska_2007-2016.pkl')
detection_cube_src = os.path.join(DATA_DIR, 'interim/modis/fire_cube/fire_detection_cube_modis_alaska_2007-2016.pkl')

integrated_cluster_df_fmt = 'interim/integrated/fire_weather/fire_weather_integrated_gfs_4_modis_5km_10days_1400_%dk_alaska_2007-2016.pkl'
integrated_cluster_df_src_list = map(lambda k: os.path.join(DATA_DIR, integrated_cluster_df_fmt % k), T_K_ARR)

# --- Load data
X_ignition_c, Y_detection_c = evm.setup_ignition_data(ignition_cube_src, detection_cube_src)

X_active_df = []
for f_src in integrated_cluster_df_src_list:
    X_active_df.append(evm.setup_active_fire_data(f_src))

In [ ]:
def no_ignition_model_poisson(covariates):
    afm = gp.GridPredictorModel(pr.PoissonRegressionModel(covariates))
    model = aig.ActiveIgnitionGridModel(afm, None)
    
    return model

def bias_model_poisson(covariates):
    afm = gp.GridPredictorModel(pr.PoissonRegressionModel(covariates))
    igm = bg.BiasGridModel()
    model = aig.ActiveIgnitionGridModel(afm, igm)
    
    return model

def bias_model_linear(covariates):
    afm = gp.GridPredictorModel(lr.LinearRegressionModel(covariates))
    igm = bg.BiasGridModel()
    model = aig.ActiveIgnitionGridModel(afm, igm)
    
    return model

def only_bias_model():
    igm = bg.BiasGridModel()
    model = aig.ActiveIgnitionGridModel(None, igm)
    
    return model

def only_zero_model():
    model = aig.ActiveIgnitionGridModel(None, None)
    
    return model

In [ ]:
def test_model(model_func):
    results_all = defaultdict(list)
    for t_k in T_K_ARR:
        results_k = {}
        
        # Test baselines
        #results = evm.evaluate_model(only_zero_model(), X_active_df[t_k-1], X_ignition_c, Y_detection_c, t_k)
        #results_all['Only_Zero'].append(results)
        
        results = evm.evaluate_model(only_bias_model(), X_active_df[t_k-1], Y_detection_c, Y_detection_c, t_k)
        results_all['Only_Bias'].append(results)
        
        # Test model with different covariates
        print 'T_k=%d' % t_k
        for name,cov in [('None', []), ('Temp/Humid', ['temperature', 'humidity']), ('All', ['temperature','humidity','wind','rain'])]:
            results = evm.evaluate_model(model_func(cov), X_active_df[t_k-1], X_ignition_c, Y_detection_c, t_k)
            results_all[name].append(results)
    
    return results_all

def flat(x):
    return map(lambda x: x.flatten(), x)

def plot_results(results):
    for metric in [metrics.mean_absolute_error, metrics.root_mean_squared_error]:
        for k,v in results.iteritems():
            plt.plot(T_K_ARR, map(lambda x: metric(*flat(x)), results[k]), "s--", label=k, linewidth=2)
        lgd = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.xlabel("Day of forecast (k)")
        plt.xticks(T_K_ARR)
        plt.ylabel(metric.__name__)
        plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [7,5]

results = test_model(no_ignition_model_poisson)
plot_results(results)

In [ ]:
results = test_model(bias_model_poisson)
plot_results(results)

In [ ]:
results = test_model(bias_model_linear)
plot_results(results)

In [ ]:
def map_grid(values, titles, fixed_minmax=False):
    values,titles = np.array(values), np.array(titles)
    shape = np.shape(values)
    
    fig = plt.figure()
    grid_spacing = [2,.5,4,.5]
    
    gmin,gmax = np.inf,-np.inf
    if fixed_minmax:
        for i in range(shape[0]):
            for j in range(shape[1]):
                lmin,lmax = np.min(values[i,j]),np.max(values[i,j])
                gmin = gmin if gmin <= lmin else lmin
                gmax = gmax if gmax >= lmax else lmax
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            t,v = titles[i,j], values[i,j]
            ax = fig.add_subplot(shape[0],shape[1],(i*shape[1])+j+1)
            ax.set_title(t)
            
            if fixed_minmax:
                vmin,vmax = gmin,gmax
            else:
                vmin,vmax = None,None
            vmap.make_contourf(vmap.make_map(BB, grid_spacing), v, BB, vmin=vmin, vmax=vmax)

In [ ]:
BB = geo.LatLonBoundingBox(55, 71, -165, -138)
plt.rcParams['figure.figsize'] = [15,10]

titles = [
    ['Mean activity per cell', 'MAE (BM_All(k=1))']
]

values = [
    [np.mean(Y_detection_c.values, axis=2), metrics.mean_absolute_error(*(results['All'][0]), axis=2)]
]
map_grid(values, titles, fixed_minmax=True)

In [ ]:
plt.rcParams['figure.figsize'] = [15,20]

titles = [
    ['MAE (BM_All(k=2 minus k=1)', 'MAE (BM_All(k=3 minus k=1)'],
    ['MAE (BM_All(k=4 minus k=1)', 'MAE (BM_All(k=5 minus k=1)']
]

k_1 = metrics.mean_absolute_error(*(results['All'][0]), axis=2)
k_2 = metrics.mean_absolute_error(*(results['All'][1]), axis=2)
k_3 = metrics.mean_absolute_error(*(results['All'][2]), axis=2)
k_4 = metrics.mean_absolute_error(*(results['All'][3]), axis=2)
k_5 = metrics.mean_absolute_error(*(results['All'][4]), axis=2)

values = [
    [k_2-k_1, k_3-k_1],
    [k_4-k_1, k_5-k_1]
]
map_grid(values, titles, fixed_minmax=True)

In [ ]:
plt.rcParams['figure.figsize'] = [15,10]

titles = [
    ['MAE (BM_All(k=1)', 'MAE (BM_None(k=1)']
]

k_1 = metrics.mean_absolute_error(*(results['All'][0]), axis=2)
k_2 = metrics.mean_absolute_error(*(results['All'][1]), axis=2)

values = [
    [metrics.mean_absolute_error(*(results['All'][0]), axis=2), metrics.mean_absolute_error(*(results['None'][0]), axis=2)]
]
map_grid(values, titles)

In [ ]:
plt.rcParams['figure.figsize'] = [15,20]

titles = [
    ['MAE (BM_None(k=1) minus BM_All(k=1))', 'MAE (BM_TempHumid(k=1) minus BM_All(k=1))']
]

bm_all_k_1 = metrics.mean_absolute_error(*(results['All'][0]), axis=2)
bm_tg_k_1 = metrics.mean_absolute_error(*(results['Temp/Humid'][1]), axis=2)
bm_none_k_1 = metrics.mean_absolute_error(*(results['None'][1]), axis=2)

values = [
    [bm_none_k_1-bm_all_k_1, bm_tg_k_1-bm_all_k_1]
]
map_grid(values, titles, fixed_minmax=True)

In [ ]:
plt.rcParams['figure.figsize'] = [15,10]
values = metrics.mean_absolute_error(*(results['Only_Bias'][0]), axis=2)
vmap.make_contourf(vmap.make_map(BB, grid_spacing), values, BB)